In [3]:
from typing import Type, Tuple

import numpy as np
import matplotlib.pyplot as plt
import nnfs
from nnfs.datasets import spiral_data, vertical_data

In [4]:
nnfs.init(0)


In [5]:
class ActivationFunction:
    def __init__(self, type: str) -> None:
        self.type = type
        


class ReLu(ActivationFunction):
    def __init__(self) -> None:
        super().__init__("Relu")
        
    def output(z: np.ndarray) -> np.ndarray:
        return np.maximum(0, z)
    
    
class Sigmoid(ActivationFunction):
    def __init__(self) -> None:
        super().__init__("Sigmoid")
        
    def output(z: np.ndarray):
        return 1 / (1 + np.exp(-z))


class Softmax(ActivationFunction):
    def __init__(self) -> None:
        super().__init__("Sigmoid")
        
    def output(z: np.ndarray):
        exp_values = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_values / np.sum(exp_values, axis=1, keepdims=True)

In [6]:
class Layer:
    def __init__(self, type: str, size: Tuple, activation: Type[ActivationFunction] | None) -> None:
        self.type: str = type
        self.size: Tuple = size
        self.activation = activation
        self.output = 0
        
    def __str__(self) -> str:
        return f"Base Layer class"

    def forward(self):
        pass

class Dense(Layer):
     
    def __init__(self, size: Tuple, activation: Type[ActivationFunction]) -> None:
        super().__init__("Dense", size, activation)
        self.weights = np.random.rand(*size) * 0.01
        self.biases = np.zeros((1, size[1]))
        
        
    def __str__(self) -> str:
        return f"- {self.type} -\nWeights:\n{self.weights}\n{'-'*15}\nBias: {self.biases}"


    def forward(self, x: np.ndarray) -> None:
        self.output = self.activation.output(np.dot(x, self.weights) + self.biases)

    

class Input(Layer):
    def __init__(self, inputs: np.ndarray) -> None:
        super().__init__("Input", inputs.size, None)
        self.output = inputs

In [33]:
class Loss:
    def __init__(self, type: str) -> None:
        self.type= type
        self.output = 0
        
    def calculate(self, output: np.ndarray, y_true: np.ndarray) -> float:
        
        self.forward(output, y_true)
        
        return np.mean(self.output)


class CategoricalCrossEntropy(Loss):
    def __init__(self) -> None:
        super().__init__("Categorical")
        
    def forward(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        
        correct_confidences = y_pred_clipped[range(len(y_pred)), y_true]
        self.output = -np.log(correct_confidences)
        

In [8]:
class Accuracy:
    def calculate(self, y_pred: np.ndarray, y_true: np.ndarray) -> float:
        
        return np.mean(np.argmax(y_pred, axis=1) == y_true)

In [9]:
class NeuralNetwork:
    def __init__(self) -> None:
        self.layers: np.ndarray[Type[Layer]] = np.array([])
    
    def __str__(self) -> str:
        return ",\n".join([f"Layer {layer.type} (size={layer.size})" for layer in self.layers])
    
    def add(self, layer: Type[Layer]) -> None:
        self.layers = np.append(self.layers, layer)
        
    def propagate_forward(self) -> np.ndarray:
        z = self.layers[0].forward()
        for layer in self.layers[1:]:
            z = layer.activation.output(layer.forward(z))
            
        print(z)  
        return CategoricalCrossEntropy().output(z, 1)

In [57]:
X, y = vertical_data(samples=100, classes=3)

dense1 = Dense(size=(2, 64), activation=ReLu)
dense2 = Dense(size=(64, 64), activation=ReLu)
dense3 = Dense(size=(64, 3), activation=Softmax)

loss_function = CategoricalCrossEntropy()

accuracy_function = Accuracy()
min_loss = 99999
max_acc = 0
best_dense1_weights = dense1.weights.copy()
best_dense1_biases = dense1.biases.copy()
best_dense2_weights = dense2.weights.copy()
best_dense2_biases = dense2.biases.copy()
best_dense3_weights = dense3.weights.copy()
best_dense3_biases = dense3.biases.copy()

In [ ]:
X, y = vertical_data(samples=100, classes=3)

dense1 = Dense(size=(2, 16), activation=ReLu)
dense2 = Dense(size=(16, 16), activation=Softmax)
dense3 = Dense(size=(16, 3), activation=Softmax)

loss_function = CategoricalCrossEntropy()

accuracy_function = Accuracy()

min_loss = 99999
max_acc = 0
best_dense1_weights = dense1.weights.copy()
best_dense1_biases = dense1.biases.copy()
best_dense2_weights = dense2.weights.copy()
best_dense2_biases = dense2.biases.copy()
best_dense3_weights = dense3.weights.copy()
best_dense3_biases = dense3.biases.copy()

for epoch in range(100000):
    
    dense1.weights += 0.05 * np.random.rand(*dense1.size)
    dense1.biases += 0.05 * np.random.rand(1, dense1.size[1])
    dense2.weights += 0.05 * np.random.rand(*dense2.size)
    dense2.biases += 0.05 * np.random.rand(1, dense2.size[1])
    dense3.weights += 0.05 * np.random.rand(*dense3.size)
    dense3.biases += 0.05 * np.random.rand(1, dense3.size[1])
    
    dense1.forward(X)
    dense2.forward(dense1.output)
    dense3.forward(dense2.output)
    loss = loss_function.calculate(dense2.output, y)
    
    
    accuracy = accuracy_function.calculate(dense2.output, y)
    
    if loss < min_loss:
        best_dense1_weights = dense1.weights.copy()
        best_dense1_biases = dense1.biases.copy()
        best_dense2_weights = dense2.weights.copy()
        best_dense2_biases = dense2.biases.copy()
        best_dense3_weights = dense3.weights.copy()
        best_dense3_biases = dense3.biases.copy()
        
        print(f"{epoch}. New values found. Loss: {loss}. Acc: {accuracy}")
        
        max_acc = accuracy
        min_loss = loss
    
    else:
        dense1.weights = best_dense1_weights.copy()
        dense1.biases = best_dense1_biases.copy()
        dense2.weights = best_dense2_weights.copy()
        dense2.biases = best_dense2_biases.copy()
        dense3.weights = best_dense3_weights.copy()
        dense3.biases = best_dense3_biases.copy()

    
    